# Multiple Linear Regression Lab : House Prices

### Overview
Estimate house prices using MLR

### Builds on
None

### Run time
approx. 20 minutes

### Notes


## Step 1 - Load Data

In [1]:
import os
import urllib.request

data_location = "house-sales-full.csv"
data_url = 'https://elephantscale-public.s3.amazonaws.com/data/house-prices/house-sales-full.csv'
#data_url = 'https://raw.githubusercontent.com/elephantscale/datasets/master/house-prices/house-sales-full.csv'

if not os.path.exists (data_location):
    data_location = os.path.basename(data_location)
    if not os.path.exists(data_location):
        print("Downloading : ", data_url)
        urllib.request.urlretrieve(data_url, data_location)
print('data_location:', data_location)


data_location: house-sales-full.csv


In [2]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

house_prices = pd.read_csv(data_location)
house_prices.sample(10)

,DocumentID,Date,SalePrice,PropertyID,PropertyType,ym,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,...,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction
20354,20355,1/11/06,765000,7511210330,Single Family,1/1/06,383200,0.88,"868,810.00",1,...,3.00,4,9,1987,0,0,270000,476000,98053,False
23168,23169,10/18/06,1110179,8562720190,Single Family,10/1/06,418300,0.96,"1,155,032.00",1,...,3.25,4,11,2006,0,0,234000,809000,98027,False
13072,13073,10/19/06,729950,3865100210,Single Family,10/1/06,418300,0.96,"759,441.00",1,...,2.50,4,9,2003,0,1,274000,522000,98008,False
10803,10804,5/11/09,463333,3307700405,Single Family,5/1/09,360700,0.83,"559,031.00",1,...,1.00,2,7,1946,0,0,486000,23000,98040,False
4078,4079,9/25/06,426000,1442860160,Single Family,9/1/06,414800,0.95,"446,951.00",1,...,2.50,3,8,1985,0,0,105000,281000,98058,False
6776,6777,4/12/07,615000,2215500205,Single Family,4/1/07,430900,0.99,"621,137.00",1,...,1.75,3,7,1945,0,1,306000,198000,98115,False
1201,1202,6/14/10,319500,452000603,Townhouse,6/1/10,345800,0.79,"402,101.00",1,...,1.50,2,8,2010,0,0,128000,292000,-1,True
18853,18854,4/29/08,504950,6979970180,Single Family,4/1/08,411700,0.95,"533,773.00",1,...,3.50,4,8,2007,0,0,129000,438000,98072,True
6544,6545,5/8/06,343000,2133000140,Single Family,5/1/06,400600,0.92,"372,625.00",1,...,1.75,3,7,1988,0,0,109000,178000,98019,False
26109,26110,9/9/11,334950,9477201670,Single Family,9/1/11,312000,0.72,"467,212.00",1,...,2.50,3,7,1975,0,0,129000,271000,98034,False


## Step 2 - Explore Data (EDA)
EDA is a great way to get a sense of the data.  

Try to find answers to the following questions, by looking at the output of `describe` below

- How many 'max' bedrooms do we have? :-)
- Find min/max of 'SalePrice'
- Do you think we have outliers in data

In [3]:
## TODO use 'describe()' function to understand data
house_prices.describe()

,DocumentID,SalePrice,PropertyID,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode
count,"27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00","27,063.00"
mean,"13,532.00","511,626.20","4,680,324,882.08","390,750.58",0.90,"570,918.76",1.02,"10,997.68","2,122.96",275.30,2.26,3.38,7.78,"1,977.09",86.73,0.21,"213,820.07","317,211.67","82,223.04"
std,"7,812.56","342,821.17","2,896,350,979.15","37,024.46",0.09,"380,236.63",0.15,"28,110.66",939.84,428.71,0.77,0.90,1.18,30.92,407.32,0.55,"177,213.41","234,038.34","36,106.67"
min,1.00,"3,000.00","1,000,102.00","311,600.00",0.72,"3,368.00",1.00,494.00,370.00,0.00,0.00,0.00,3.00,"1,900.00",0.00,0.00,0.00,0.00,-1.00
25%,"6,766.50","329,000.00","2,213,000,057.50","357,100.00",0.82,"366,918.50",1.00,"4,257.50","1,440.00",0.00,1.75,3.00,7.00,"1,954.00",0.00,0.00,"105,000.00","183,000.00","98,019.00"
50%,"13,532.00","425,000.00","3,972,900,140.00","400,600.00",0.92,"475,664.00",1.00,"6,636.00","1,940.00",0.00,2.50,3.00,8.00,"1,986.00",0.00,0.00,"172,000.00","261,000.00","98,053.00"
75%,"20,297.50","590,000.00","7,504,001,385.00","421,200.00",0.97,"655,061.00",1.00,"9,450.00","2,610.00",510.00,2.50,4.00,8.00,"2,006.00",0.00,0.00,"258,000.00","380,000.00","98,115.00"
max,"27,063.00","11,000,000.00","9,906,000,035.00","435,200.00",1.00,"11,644,855.00",5.00,"1,024,068.00","10,740.00","3,500.00",8.00,33.00,13.00,"2,016.00","2,016.00",3.00,"5,538,000.00","5,772,000.00","98,354.00"


In [4]:
## any correlated columns?
## TODO : use 'corr' function to verify
house_prices.corr()

,DocumentID,SalePrice,PropertyID,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction
DocumentID,1.00,-0.02,0.99,-0.02,-0.02,-0.02,-0.02,-0.11,-0.02,-0.03,-0.01,-0.03,-0.01,0.03,-0.01,-0.04,-0.03,-0.01,-0.01,0.05
SalePrice,-0.02,1.00,-0.02,0.12,0.12,0.98,0.02,0.14,0.68,0.29,0.52,0.32,0.66,0.08,0.09,-0.01,0.79,0.81,-0.02,0.04
PropertyID,0.99,-0.02,1.00,-0.02,-0.02,-0.02,-0.01,-0.11,-0.02,-0.03,-0.01,-0.03,-0.01,0.03,-0.01,-0.04,-0.03,-0.02,-0.01,0.05
zhvi_px,-0.02,0.12,-0.02,1.00,1.00,-0.03,0.02,-0.00,-0.01,0.02,-0.03,0.00,-0.04,-0.06,0.01,0.01,-0.00,-0.02,0.09,-0.02
zhvi_idx,-0.02,0.12,-0.02,1.00,1.00,-0.03,0.02,-0.00,-0.01,0.02,-0.03,0.00,-0.04,-0.06,0.01,0.01,-0.00,-0.02,0.09,-0.02
AdjSalePrice,-0.02,0.98,-0.02,-0.03,-0.03,1.00,0.02,0.14,0.69,0.29,0.52,0.32,0.67,0.08,0.09,-0.02,0.80,0.82,-0.03,0.04
NbrLivingUnits,-0.02,0.02,-0.01,0.02,0.02,0.02,1.00,-0.00,0.05,0.06,0.10,0.16,-0.05,-0.11,0.01,0.08,0.03,-0.00,0.04,-0.05
SqFtLot,-0.11,0.14,-0.11,-0.00,-0.00,0.14,-0.00,1.00,0.21,0.05,0.10,0.08,0.14,0.03,0.01,-0.01,0.08,0.11,0.06,-0.09
SqFtTotLiving,-0.02,0.68,-0.02,-0.01,-0.01,0.69,0.05,0.21,1.00,0.39,0.75,0.62,0.76,0.30,0.05,-0.07,0.47,0.76,-0.10,0.11
SqFtFinBasement,-0.03,0.29,-0.03,0.02,0.02,0.29,0.06,0.05,0.39,1.00,0.26,0.30,0.14,-0.22,0.11,0.05,0.29,0.28,0.10,-0.14


## Step 3 - Shape Data
Wow! That's a lot of columns.  Maybe we should foucs on just a few of them.

**=> Select only "SalePrice", "Bedrooms", "Bathrooms", "SqFtTotLiving", "SqFtLot"**

In [44]:
## TODO : select columns
## start with : 'Bedrooms', 'Bathrooms', 'SqFtTotLiving', 'SqFtLot'
input_columns= ['SqFtTotLiving','LandVal','ImpsVal','BldgGrade']

## TODO : select label column
##  Hint : 'SalePrice'
label_column = ['SalePrice']

house_prices2 = house_prices[input_columns + label_column]
house_prices2

,SqFtTotLiving,LandVal,ImpsVal,BldgGrade,SalePrice
0,2400,70000,229000,7,280000
1,3764,203000,590000,10,1000000
2,2060,183000,275000,8,745000
3,3200,104000,229000,7,425000
4,1720,104000,205000,7,240000
...,...,...,...,...,...
27058,1410,147000,269000,9,374000
27059,1410,147000,269000,9,374000
27060,1070,92000,130000,6,165000
27061,1345,103000,212000,7,315000


In [45]:
x = house_prices2[input_columns]
y = house_prices2[label_column]

print ("x.shape = ", x.shape)
print ("y.shape = ", y.shape)

x.shape =  (27063, 4)
y.shape =  (27063, 1)


## Step 4 : Run Regression

In [46]:
from sklearn.linear_model import LinearRegression

##  algo + data --> model
algo = LinearRegression ()
model = algo.fit(x, y)  ## TODO : fit (x,y)
model

LinearRegression()

## Step 5 : Predict

In [47]:
## TODO Predict on 'x'
predictions = model.predict(x)
predictions

array([[319969.56],
       [760914.15],
       [458445.53],
       ...,
       [217302.24],
       [303413.33],
       [513185.49]])

In [48]:
pd.options.display.float_format = '{:,.2f}'.format

a = house_prices2[input_columns  + label_column]
a['predicted_price'] = predictions
a

,SqFtTotLiving,LandVal,ImpsVal,BldgGrade,SalePrice,predicted_price
0,2400,70000,229000,7,280000,"319,969.56"
1,3764,203000,590000,10,1000000,"760,914.15"
2,2060,183000,275000,8,745000,"458,445.53"
3,3200,104000,229000,7,425000,"380,208.86"
4,1720,104000,205000,7,240000,"313,375.32"
...,...,...,...,...,...,...
27058,1410,147000,269000,9,374000,"414,451.34"
27059,1410,147000,269000,9,374000,"414,451.34"
27060,1070,92000,130000,6,165000,"217,302.24"
27061,1345,103000,212000,7,315000,"303,413.33"


## Step 6 : Evaluate

**Q==> Are any coefficients close to zero?  What does that mean?**

**Q==> Also inspect R2 value.  Is it decent?**

In [49]:
## coefficients
import numpy as np
np.set_printoptions(precision=2, suppress=True)

print ("coefficients: " , model.coef_)
print ("intercept : ", model.intercept_)

coefficients:  [[   35.35     0.94     0.6  16451.84]]
intercept :  [-84343.5]


In [50]:
## print each feature and it's coefficients
coef = pd.DataFrame({"input_column" : input_columns,  
                     "coefficient": model.coef_[0]})
coef

,input_column,coefficient
0,SqFtTotLiving,35.35
1,LandVal,0.94
2,ImpsVal,0.60
3,BldgGrade,"16,451.84"


In [51]:
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

## TODO : find R2 by comparing 'y' and 'predictions'
print ("R2 : " , r2_score(y, predictions))

print ("MSE : ", mean_squared_error(y, predictions))
print ("RMSE : ", sqrt(mean_squared_error(y, predictions)) )

R2 :  0.8231542037276159
MSE :  20783273624.81069
RMSE :  144164.05108351627


## Step 7 : Predict on New Data

In [ ]:
## Create some of your own data
## Bedrooms : 4,  Bathrooms : 3.5,   SqFtTotLiving : 2000, SqFtLot: 3000
new_data = pd.DataFrame({'Bedrooms' : [5,3,???],
                         'Bathrooms' : [3,2, ???],
                         'SqFtTotLiving' : [4400, 1800, ???],
                         'SqFtLot' : [10000, 5000, ???]
             })
new_data

In [ ]:
new_prediction = model.predict(new_data)
new_prediction

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

new_data['predicted_price'] = new_prediction
new_data

## Step 8 : Imporove Model Performance
Now that we have done a 'end-to-end' implementation in Regression.  
How ever our accuracy isn't all that great!  

**Q ==> What can we do to improve our model?**

One option is to choose better input columns.  
in Step-3, add more input columns.
For example you can add 'LandVal' to input columns as follows

```python
input_columns= ['Bedrooms', 'Bathrooms', 'SqFtTotLiving', 'SqFtLot', 'LandVal']
```

And then run the notebook again, and see if the accuracy goes up!

### Class Competition

Let's see how high you can bring up the model accuracy, with fewest columns as possible.